<a href="https://colab.research.google.com/github/GajeraKishan/NaturalLanguageInference/blob/master/NLPProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install mecab-python3

In [2]:
!pip install fairseq

     |████████████████████████████████| 307kB 10.5MB/s 
     |████████████████████████████████| 61kB 8.9MB/s 
     |████████████████████████████████| 17.1MB 203kB/s 
  Created wheel for fairseq: filename=fairseq-0.9.0-cp36-cp36m-linux_x86_64.whl size=2015421 sha256=b1f3fcbaab5c5270dd90f033b6152e3ef50d61b6b0093bfb22c763fe282f4473
  Stored in directory: /root/.cache/pip/wheels/37/3e/1b/0fa30695dcba41e4b0088067fa40f3328d1e8ee78c22cd4766
Successfully built fairseq


In [5]:
!gdown 'https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/encoder.json'

Downloading...
From: https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/encoder.json
To: /content/encoder.json
1.04MB [00:00, 1.08MB/s]


In [6]:
!gdown 'https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/vocab.bpe'

Downloading...
From: https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/vocab.bpe
To: /content/vocab.bpe
456kB [00:00, 638kB/s]


In [13]:
!bash python multiprocessing_bpe_encoder.py --encoder-json encoder.json --vocab-bpe vocab.bpe --inputs "sentences_train.txt" --outputs "train.input0.bpe" --workers 1 --keep-empty

processed 10000 lines
processed 20000 lines
processed 30000 lines
processed 40000 lines
processed 50000 lines
processed 60000 lines


In [8]:
!bash python multiprocessing_bpe_encoder.py --encoder-json encoder.json --vocab-bpe vocab.bpe --inputs "sentences_val.txt" --outputs "dev.input0.bpe" --workers 1 --keep-empty

processed 10000 lines


In [9]:
!gdown 'https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/dict.txt'

Downloading...
From: https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/dict.txt
To: /content/dict.txt
603kB [00:00, 788kB/s]


In [10]:
!fairseq-preprocess --only-source --trainpref "train.input0.bpe" --validpref "dev.input0.bpe" --destdir "train-bin/input0" --workers 60 --srcdict dict.txt

Namespace(align_suffix=None, alignfile=None, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='train-bin/input0', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=1000, lr_scheduler='fixed', memory_efficient_fp16=False, min_loss_scale=0.0001, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=True, optimizer='nag', padding_factor=8, seed=1, source_lang=None, srcdict='dict.txt', target_lang=None, task='translation', tensorboard_logdir='', testpref=None, tgtdict=None, threshold_loss_scale=None, thresholdsrc=0, thresholdtgt=0, tokenizer=None, trainpref='train.input0.bpe', user_dir=None, validpref='dev.input0.bpe', workers=60)
| [None] Dictionary: 50263 types
| [None] train.input0.bpe: 60000 sents, 7734940 tokens, 0.0% replaced by <unk>
| [None] Dictionary: 50263 types
| [None] dev.input0.bpe: 10000 sents, 1192525 tokens, 0.0% replaced by <unk

In [12]:
!fairseq-preprocess --only-source --trainpref "labels_train.txt" --validpref "labels_val.txt" --destdir "train-bin/label" --workers 60

Namespace(align_suffix=None, alignfile=None, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='train-bin/label', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=1000, lr_scheduler='fixed', memory_efficient_fp16=False, min_loss_scale=0.0001, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=True, optimizer='nag', padding_factor=8, seed=1, source_lang=None, srcdict=None, target_lang=None, task='translation', tensorboard_logdir='', testpref=None, tgtdict=None, threshold_loss_scale=None, thresholdsrc=0, thresholdtgt=0, tokenizer=None, trainpref='labels_train.txt', user_dir=None, validpref='labels_val.txt', workers=60)
| [None] Dictionary: 7 types
| [None] labels_train.txt: 60000 sents, 120000 tokens, 0.0% replaced by <unk>
| [None] Dictionary: 7 types
| [None] labels_val.txt: 10000 sents, 20000 tokens, 0.0% replaced by <unk>
| Wrote preproce

In [14]:
!bash  python train.py train-bin/ \
    --restore-file model.pt \
    --max-positions 512 \
    --max-sentences 2 \
    --max-tokens 4400 \
    --task sentence_prediction \
    --reset-optimizer --reset-dataloader --reset-meters \
    --required-batch-size-multiple 1 \
    --init-token 0 --separator-token 2 \
    --arch roberta_large \
    --criterion sentence_prediction \
    --num-classes 2 \
    --dropout 0.1 --attention-dropout 0.1 \
    --weight-decay 0.1 --optimizer adam --adam-betas "(0.9, 0.98)" --adam-eps 1e-04 \
    --clip-norm 0.0 \
    --lr-scheduler polynomial_decay --lr 0.00001 --total-num-update 10000 --warmup-updates 50 \
    --fp16 --fp16-init-scale 4 --threshold-loss-scale 1 --fp16-scale-window 128 \
    --max-epoch 5 \
    --best-checkpoint-metric accuracy --maximize-best-checkpoint-metric \
    --truncate-sequence \
    --find-unused-parameters \
    --update-freq 16

Namespace(activation_dropout=0.0, activation_fn='gelu', adam_betas='(0.9, 0.98)', adam_eps=0.0001, add_prev_output_tokens=False, arch='roberta_large', attention_dropout=0.1, best_checkpoint_metric='accuracy', bpe=None, bucket_cap_mb=25, clip_norm=0.0, cpu=False, criterion='sentence_prediction', curriculum=0, data='train-bin/', dataset_impl=None, ddp_backend='c10d', device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_port=-1, distributed_rank=0, distributed_world_size=1, dropout=0.1, empty_cache_freq=0, encoder_attention_heads=16, encoder_embed_dim=1024, encoder_ffn_embed_dim=4096, encoder_layerdrop=0, encoder_layers=24, encoder_layers_to_keep=None, end_learning_rate=0.0, fast_stat_sync=False, find_unused_parameters=True, fix_batches_to_gpus=False, fixed_validation_seed=None, force_anneal=None, fp16=True, fp16_init_scale=4, fp16_scale_tolerance=0.0, fp16_scale_window=128, init_token=0, keep_interval_upd